# CNN Visualization
1. Activation maps
2. Filter using gradient ascent
3. Deep dream using gradient ascent

In [1]:
%env CUDA_VISIBLE_DEVICES = ""

env: CUDA_VISIBLE_DEVICES=""


In [2]:
import os 
import tensorflow as tf 
import tensorflow.keras as keras
tf.config.gpu.set_per_process_memory_growth(True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
tf.compat.v1.disable_eager_execution()

Use a pretrained model like VGG16 and VGG19 trained on ImageNet to do the task.

In [3]:
vgg19 = keras.applications.VGG19(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [4]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
export_path = './model'
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

# Build the signature_def_map.
inputs = tf.compat.v1.saved_model.build_tensor_info(vgg19.input)
add_variable = False
for l in vgg19.layers[1:]:
    l_t = tf.compat.v1.saved_model.build_tensor_info(l.output)
    signature = (
        # method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME
        tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
            inputs={'input': inputs},
            outputs={'output': l_t},
            method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
        )
    )

    if add_variable:
        builder.add_meta_graph([tf.compat.v1.saved_model.tag_constants.SERVING],
            signature_def_map={l.name:signature,},
            strip_default_attrs=True
        )
    else:
        builder.add_meta_graph_and_variables(
            tf.compat.v1.keras.backend.get_session(), [tf.compat.v1.saved_model.tag_constants.SERVING],
            signature_def_map={l.name:signature,},
            strip_default_attrs=True
        )
        add_variable = False
builder.save()

W0708 11:36:29.641474 140463972824896 deprecation.py:323] From <ipython-input-6-dd55930ffc18>:5: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


AssertionError: Graph state including variables and assets has already been saved. Please invoke `add_meta_graph()` instead.